In [1]:
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('Processed_Data.csv', engine = 'python')
df

,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275,Positive,aussie would happy series happen midst olympic...
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.13333333333333333,Negative,worst thing olympics finish whole week availab...
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.0,Neutral,olympics play india hockey captain ranirampal ...
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.0,Positive,see best moment tokyo close ceremony videoelep...
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.5,Positive,fabulous olympics lovethebbc tokyo olympics bb...
...,...,...,...,...,...,...
140253,140242,Tokyo olympics,Congratulations to all our winners and partici...,0.5,Positive,congratulation winner participant olympics win...
140254,140243,Tokyo olympics,I hope that I am wrong but I have seen no twee...,-0.5,Negative,hope wrong see tweet government
140255,140244,Tokyo olympics,Tokyo passes the baton to Paris as strangest e...,0.0,Neutral,tokyo pass baton paris strangest ever olympic ...
140256,140245,Tokyo olympics,"Paris plans to deliver inclusive, youth-centre...",0.0,Neutral,paris plan deliver inclusive youth centre gend...


Downloading the Pre-trained GLove File's

In [3]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2021-10-01 07:16:18--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-10-01 07:16:18--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-10-01 07:16:19--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [5]:
df = df.dropna()
df.shape

(139707, 6)

In [6]:
def ratio(x):
  if x=='Positive':
    return 1
  elif x=='Negative':
    return 2
  else:
    return 0;

In [7]:
df['label'] = df['class'].apply(ratio)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,topic,tweet,sentiment,class,processed_tweets,label
0,0,#olympics,Aussies would be happy that the T20 series hap...,0.275,Positive,aussie would happy series happen midst olympic...,1
1,1,#olympics,The worst thing about the #Olympics finishing ...,-0.13333333333333333,Negative,worst thing olympics finish whole week availab...,2
2,2,#olympics,#Olympics\n\nWe play for India: #Hockey captai...,0.0,Neutral,olympics play india hockey captain ranirampal ...,0
3,3,#olympics,See the best moments from the #Tokyo2020 closi...,1.0,Positive,see best moment tokyo close ceremony videoelep...,1
4,4,#olympics,Fabulous! #Olympics \n#LoveTheBBC \n\nTokyo Ol...,0.5,Positive,fabulous olympics lovethebbc tokyo olympics bb...,1


In [8]:
df.processed_tweets = df.processed_tweets.astype(str)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
y = df['label']

In [14]:
x = df['processed_tweets']

In [10]:
reviews = df['processed_tweets']
reviews

0         aussie would happy series happen midst olympic...
1         worst thing olympics finish whole week availab...
2         olympics play india hockey captain ranirampal ...
3         see best moment tokyo close ceremony videoelep...
4         fabulous olympics lovethebbc tokyo olympics bb...
                                ...                        
140253    congratulation winner participant olympics win...
140254                      hope wrong see tweet government
140255    tokyo pass baton paris strangest ever olympic ...
140256    paris plan deliver inclusive youth centre gend...
140257    tokyo game could yet positive legacy despite u...
Name: processed_tweets, Length: 139707, dtype: object

In [15]:
X_train, X_test,Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state = 45)

In [16]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(111765,)
(27942,)
(111765,)
(27942,)


In [17]:
len(Y_train)

111765

In [18]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [19]:
words_to_index = tokenizer.word_index
len(words_to_index)

37826

In [20]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [21]:
word_to_vec_map = read_glove_vector('glove.6B.50d.txt')

In [22]:
maxLen = 150
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index-1, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

In [23]:
embedding_layer

In [24]:
def conv1d_model(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = Conv1D(512,3,activation='relu')(embeddings)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  
  X = MaxPooling1D(3)(X)

  X = Conv1D(256,3,activation='relu')(X)
  X = Dropout(0.8)(X)
  X = MaxPooling1D(3)(X)

  X = GlobalMaxPooling1D()(X)

  X = Dense(256, activation='relu')(X)
  X = Dense(3, activation='softmax')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [25]:
model_1d = conv1d_model((maxLen,))
model_1d.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           1891300   
_________________________________________________________________
conv1d (Conv1D)              (None, 148, 512)          77312     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 49, 512)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 47, 256)           393472    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 15, 256)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 256)           196864

In [26]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

In [27]:
X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')
X_train_indices.shape

(111765, 150)

In [32]:
from tensorflow.keras import optimizers

In [30]:
from tensorflow.keras.optimizers import Adam

In [38]:
import tensorflow as tf

In [39]:
recall = tf.keras.metrics.Recall()
precision = tf.keras.metrics.Precision()

In [40]:

model_1d.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [41]:
model_1d.fit(X_train_indices, Y_train, batch_size=64, epochs=10)


Epoch 1/10
1747/1747 [==============================] - 398s 227ms/step - loss: 0.8356 - accuracy: 0.6164
Epoch 2/10
1747/1747 [==============================] - 400s 229ms/step - loss: 0.6708 - accuracy: 0.7205
Epoch 3/10
1747/1747 [==============================] - 398s 228ms/step - loss: 0.5740 - accuracy: 0.7713
Epoch 4/10
1747/1747 [==============================] - 397s 227ms/step - loss: 0.5032 - accuracy: 0.8045
Epoch 5/10
1747/1747 [==============================] - 399s 228ms/step - loss: 0.4493 - accuracy: 0.8282
Epoch 6/10
1747/1747 [==============================] - 399s 229ms/step - loss: 0.3968 - accuracy: 0.8529
Epoch 7/10
1747/1747 [==============================] - 396s 227ms/step - loss: 0.3545 - accuracy: 0.8705
Epoch 8/10
1747/1747 [==============================] - 396s 227ms/step - loss: 0.3144 - accuracy: 0.8871
Epoch 9/10
1747/1747 [==============================] - 398s 228ms/step - loss: 0.2810 - accuracy: 0.9008
Epoch 10/10
1747/1747 [=======================

In [42]:
X_test_indices = tokenizer.texts_to_sequences(X_test)
X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [43]:
model_1d.evaluate(X_test_indices, Y_test)

874/874 [==============================] - 28s 32ms/step - loss: 0.5487 - accuracy: 0.7955


[0.5487364530563354, 0.7955407500267029]